In [1]:
import numpy as np
import pandas as pd
import statistics
from openpyxl import load_workbook
from openpyxl.utils import get_column_letter, column_index_from_string
import random
from datetime import datetime
import seaborn as sns
import datetime
import os

pd.set_option('max_rows', 10000)
pd.set_option('max_columns', 100)

In [2]:
# переписывает формат файла со старого на новый. Корректно работает с '.xls' -> '.xlsx', с другими не проверялось
# есть возможность удалять обработанный файл, чтобы не иметь дубликатов (прим.: file.xls и file.xlsx)
# для этого ставим remove=True, изначально удаление выключено

# тут функция немного изменена, чтобы считывать и сохранять файлы из разных директорий

def rewrite_format(file, old_format='.xls', new_format='.xlsx', path_raw=os.getcwd(), path_processed=os.getcwd(), remove=False):
    os.chdir(path_raw)
    df = pd.read_excel(file, index_col=0, header=None)
    os.chdir(path_processed)
    df.to_excel(file[:len(file)-len(old_format)]+new_format)
    os.chdir(path_raw)
    if remove:
        os.remove(file)

In [3]:
# заносит в список всё, что format (прим: при format='.xlsx' добавит все файлы, которые в формате .xlsx)

def remove_formats(processing_list, format='.xlsx'): # надо переименовать функцию, оно не удоляет
    list_to_delete = []
    for file in processing_list:
        if format not in file:
            list_to_delete.append(file)
    for file in list_to_delete:
        processing_list.remove(file)
    list_to_delete = []
    
    # исключение для формата '.xls', т.к. при его поиске можно захватить формат '.xlsx' в подарок
    
    if format == '.xls':
        for file in processing_list:
            if '.xlsx' in file:
                list_to_delete.append(file)
        for file in list_to_delete:
            processing_list.remove(file)
    
    return processing_list

In [4]:
def pererabotka_detey(members, birth_year): # OMNOMNOMNOMNOMNOMNOM
    if len(members) == 0:
        return 0,\ # кол-во детей
               0,\ # минимальный возраст детей
               0,\ # средний возраст детей
               0   # максимальный возраст детей
    
    ch_clean, yr_clean = [], [] # создаём пустые списки для детей и их возрастов
    
    for i in list(range(len(members))): # добавляем в списки только детей и их возраста
        if members[i] == 'Дочь' or members[i] == 'Сын':
            ch_clean.append(members[i]), yr_clean.append(birth_year[i])
            
    if len(ch_clean) > 0:
        if None in yr_clean: # дети есть, возрастов нет
            return len(ch_clean),\ # кол-во детей
                   0,\             # минимальный возраст детей
                   0,\             # средний возраст детей
                   0               # максимальный возраст детей
        else:
            return len(ch_clean),\                                                        # кол-во детей
                   date_to_dict(current_date())['y'] - min(yr_clean),\                    # минимальный возраст детей
                   date_to_dict(current_date())['y'] - round(statistics.mean(yr_clean)),\ # средний возраст детей
                   date_to_dict(current_date())['y'] - max(yr_clean)                      # максимальный возраст детей
        
    elif len(ch_clean) == 0:
        return 0,\ # кол-во детей
               0,\ # минимальный возраст детей
               0,\ # средний возраст детей
               0   # максимальный возраст детей

In [5]:
def fio_enhance(name, last_name, batya_name=None): # соединяес имя, фамилию и отчество (если есть)
    try:
        if batya_name == None:
            return name + ' ' + last_name
        else:
            return name + ' ' + last_name + ' ' + batya_name
    except KeyError:
        return name + ' ' + last_name

In [6]:
def current_date(): # устанавливает актуальную дату в зависимости от года создания анкет
    if os.getcwd().split('\\')[-1] in ['fired_2018', 'fired_processed_2018']:
        return '31.12.2018'
    elif os.getcwd().split('\\')[-1] in ['raw', 'processed']:
        return '24.01.2022'
    elif os.getcwd().split('\\')[-1] in ['fired_2019', 'fired_processed_2019']:
        return '31.12.2019'
    elif os.getcwd().split('\\')[-1] in ['fired_2020', 'fired_processed_2020']:
        return '31.12.2020'
    elif os.getcwd().split('\\')[-1] in ['fired_2021', 'fired_processed_2021']:
        return '31.12.2021'

In [7]:
def date_to_str(date): # возвращает словарь вида {'y':int(),'m':int(),'d':int()}
    dates_dict = {"января":'01', "февраля":'02', "марта":'03', "апреля":'04',"мая":'05', "июня":'06', 
                  "июля":'07', "августа":'08', "сентября":'09', "октября":'10', "ноября":'11', "декабря":'12'}
    date_new = date.split()[:3]
    for time in date_new:
        if time in dates_dict:
            date_new[1] = str(dates_dict[time])
    return '.'.join(date_new)

In [8]:
def date_to_dict(date):
    return {'d':int(date.split('.')[0]), 'm':int(date.split('.')[1]), 'y':int(date.split('.')[2])}

In [9]:
def date_delta(date_big, date_smol): # возвращает разницу двух дат в количестве дней
    if date_smol == None:
        return 0
    else:
        date1, date2 = date_to_dict(date_big), date_to_dict(date_smol)
        return (date1['y'] - date2['y']) * 365 + (date1['m'] - date2['m']) * 30 + (date1['d'] - date2['d'])

In [10]:
def date_age(date_big, age): # вычисление возраста (год анкеты - год рождения)
    date1, date2 = date_to_dict(date_big), date_to_dict(age)
    return date1['y'] - date2['y']

In [11]:
def make_empty_dict(): # шаблон для построения датафрейма
    return {'Пол':None,
             'ФИО':None,
             'Возраст':None,
             'Стаж (в днях)':None,
             'Образование':None,
             'Наименование образовательного учреждения':None,
             'Квалификация по документу об образовании':None,
             'Направление или специальность по документу':None,
             'Должность':None,
             'Отдел':None,
             'Должность занималась, дней':None,
             'Состояние в браке':None,
             'Количество детей':None,
             'Минимальный возраст детей':None,
             'Средний возраст детей':None,
             'Максимальный возраст детей':None,
             'Адрес места жительства':None,
             'Дата увольнения':None,
             }

In [12]:
def make_missing_keys(dick): # создаём недостающие key, иначе будет ашыпка, когда будем
                             # запихивать всё енто в таблицу
    list_of_names = ['Пол\n(мужской,\nженский)', '2. Дата рождения', 'Трудовой договор', '6. Образование', 
                 'Наименование образовательного учреждения', 'Квалификация по документу об образовании', 
                 'Направление или специальность по документу', '9. Состояние в браке', 'Фактический', 'Дата увольнения']
    for name in list_of_names:
        if name not in list(dick.keys()):
            dick[name] = None
    return dick

In [13]:
# ставим рабочие папочки

path_raw, path_processed, path_fired_2018, path_fired_processed_2018, path_fired_2021, path_fired_processed_2021 = \
    'C:/Users/Zhilapv/Desktop/HR_govno/raw',\
    'C:/Users/Zhilapv/Desktop/HR_govno/processed',\
    'C:/Users/Zhilapv/Desktop/HR_govno/fired_2018',\
    'C:/Users/Zhilapv/Desktop/HR_govno/fired_processed_2018',\
    'C:/Users/Zhilapv/Desktop/HR_govno/fired_2021',\
    'C:/Users/Zhilapv/Desktop/HR_govno/fired_processed_2021'
path_fired_2019, path_fired_processed_2019, path_fired_2020, path_fired_processed_2020 = \
    'C:/Users/Zhilapv/Desktop/HR_govno/fired_2019',\
    'C:/Users/Zhilapv/Desktop/HR_govno/fired_processed_2019',\
    'C:/Users/Zhilapv/Desktop/HR_govno/fired_2020',\
    'C:/Users/Zhilapv/Desktop/HR_govno/fired_processed_2020'
os.chdir(path_fired_2018)

# считываем все файлы из рабочей папочки

xls_files = remove_formats(os.listdir(), format='.xls')
xls_files

In [15]:
# сделано, закоментированно
os.chdir(path_fired_2020)
xls_files = remove_formats(os.listdir(), format='.xls')
# обрабатываем сырые .xls файлы, приводим в формат 21 века

for file in xls_files:
    rewrite_format(file, path_raw=path_fired_2020, path_processed=path_fired_processed_2020)

In [14]:
list_of_all = [['Пол\n(мужской,\nженский)', ['down'], [int(1)]], # AG10 вниз 1
               ['1. Фамилия', ['right'], [int(1)]], # A18 вправо 1
               ['Имя', ['right'], [int(3)]], # K18 вправо 3
               ['Отчество', ['right'], [int(3)]], # W18 вправо 3
               ['2. Дата рождения', ['right'], [int(2)]], # A20 вправо 2
               ['Трудовой договор', ['down', 'right'], [int(1), int(10)]], # W16 вниз 1, вправо 10
               ['6. Образование', ['right'], [int(2)]], # A30 вправо 2
               ['Наименование образовательного учреждения', ['down'], [int(1)]], # A33 вниз 1
               #['Год окончания', ['down'], [int(1)]], # Z33 вниз 1
               ['Квалификация по документу об образовании', ['down'], [int(1)]], # A36 вниз 1
               ['Направление или специальность по документу', ['down'], [int(1)]], # N36 вниз 1
               ['8. Стаж работы', ['down'], [int(1)]], # A68 вниз 1, целиком страну
               ['9. Состояние в браке', ['right'], [int(3)]], # A74 вправо 3
               ['10. Состав семьи:', ['down'], [int(3), int(6)]], # A76 вниз 3:вниз 6
               #['Год рождения', ['down'], [int(3), int(6)]], # AE77 вниз 3:вниз 6
               #['12. Адрес места жительства:', ['down', 'right'], [int(3), int(6)]], # A93 вниз 6, вправо 9
               ['Фактический', ['right'], [int(8)]],
               ['Структурное\nподразделение', [], []], # A126 вправо 6, (вниз 4:вниз 9)
               #['Должность (специальность, профессия), разряд, класс (категория) квалификации', [], []], 
               ['Дата увольнения', ['right'], [int(2)]] ]
               #['XI. Основание прекращения трудового']] # A216 ___

# обработка уволенных 2018

In [33]:
# ставим рабочую директорию, захватываем нужные файлы
os.chdir(path_fired_processed_2020)
files = remove_formats(os.listdir(), format='.xlsx')

# делаем пустой датафрейм по шаблону
to_df = make_empty_dict()
df_fired_2020 = pd.DataFrame(columns=list(to_df.keys()))

# поiхали
for _ in files:
    wb = load_workbook(_)
    sheet = wb.active
    print(_)
    employee = {}
    childrens, birth_year, structure, date, position = [], [], [], [], []
    for row in sheet['A1':'AG300']: # движение по строкам в диапазоне ['A1':'AG300']
        for cell in row: # движение по ячейкам в строке row
            for ex in list_of_all: # перебор значений из списка того, что нужно выдрать из анкет
                if cell.value == ex[0]: # сравнение значения ячейки с названием нужного "столбца"
                    if len(ex[1]) == 1: # проверяем "направление" на движение в одной плоскости
                        if ex[1][0] == 'down': # от целевой ячейки движение вниз на ex[2][0]
                            if sheet.cell(row=cell.row+ex[2][0], column=cell.col_idx).value == None:
                                break
                            employee[str(ex[0])] = sheet.cell(row=cell.row+ex[2][0], column=cell.col_idx).value
                        elif ex[1][0] == 'right': # от целевой ячейки движение вправо на ex[2][0]
                            if sheet.cell(row=cell.row, column=cell.col_idx+ex[2][0]).value == None:
                                break
                            employee[str(ex[0])] = sheet.cell(row=cell.row, column=cell.col_idx+ex[2][0]).value
                    elif len(ex[1]) > 1: # от целевой клетки движение на ex[2][0] вниз и на ex[2][1] вправо
                        if sheet.cell(row=cell.row+ex[2][0], column=cell.col_idx+ex[2][1]).value == None:
                            break
                        employee[str(ex[0])] = sheet.cell(row=cell.row+ex[2][0], column=cell.col_idx+ex[2][1]).value
                    if '10. Состав семьи:' in cell.value: # исключение для детишек, просто ебанутый случай, нужно выдрать list[]
                        for i in list(range(1,7)):
                            if sheet.cell(row=cell.row+2+i, column=cell.col_idx).value == None:
                                break
                            childrens.append(sheet.cell(row=cell.row+2+i, column=cell.col_idx).value)
                            birth_year.append(sheet.cell(row=cell.row+2+i, column=cell.col_idx+30).value)
                    if 'Структурное\nподразделение' in cell.value: # высасываем отдел-должность в list[]
                        for i in list(range(10)):
                            if sheet.cell(row=cell.row+i+2, column=cell.col_idx).value == None:
                                break
                            structure.append(sheet.cell(row=cell.row+i+2, column=cell.col_idx).value)
                            date.append(sheet.cell(row=cell.row+i+2, column=cell.col_idx-1).value)
                            position.append(sheet.cell(row=cell.row+i+2, column=cell.col_idx+5).value)
    employee.update({'childrens':childrens, 'birth_year':birth_year, 'structure':structure, 'date':date,'position':position,
                     '2. Дата рождения':date_to_str(employee['2. Дата рождения']),
                      'Дата увольнения': date_to_str(employee['Дата увольнения'])})
       
    try:
        fio = fio_enhance(employee['1. Фамилия'], employee['Имя'], employee['Отчество'])
    except KeyError:
        fio = fio_enhance(employee['1. Фамилия'], employee['Имя'])
    employee['fio'] = fio
    employee = make_missing_keys(employee)
    
    for i in range(len(employee['structure'])):
        to_df.update({'Пол':[employee['Пол\n(мужской,\nженский)']],
                 'ФИО':[employee['fio']],
                 'Возраст':[date_age(current_date(), employee['2. Дата рождения'])],
                 'Стаж (в днях)':[date_delta(employee['Дата увольнения'], employee['Трудовой договор'])],
                 'Образование':[employee['6. Образование']],
                 'Наименование образовательного учреждения':[employee['Наименование образовательного учреждения']],
                 'Квалификация по документу об образовании':[employee['Квалификация по документу об образовании']],
                 'Направление или специальность по документу':[employee['Направление или специальность по документу']],
                 'Должность':[employee['position'][i]],
                 'Отдел':[employee['structure'][i]],
                 'Должность занималась, дней':[date_delta(date_to_str(employee['Дата увольнения']), date_to_str(employee['date'][i]))],
                 'Состояние в браке':[employee['9. Состояние в браке']],
                 'Количество детей':[pererabotka_detey(employee['childrens'], employee['birth_year'])[0]],
                 'Минимальный возраст детей':[pererabotka_detey(employee['childrens'], employee['birth_year'])[1]],
                 'Средний возраст детей':[pererabotka_detey(employee['childrens'], employee['birth_year'])[2]],
                 'Максимальный возраст детей':[pererabotka_detey(employee['childrens'], employee['birth_year'])[3]],
                 'Адрес места жительства':[employee['Фактический']],
                 'Дата увольнения':[date_to_str(employee['Дата увольнения'])],
                 })

        to_append = pd.DataFrame(data=to_df)
        df_fired_2020 = pd.concat([df_fired_2020, to_append], ignore_index=True, axis=0)
df_fired_2020['Уволен (1 да, 0 нет)'] = 1
df_fired_2020

Личная карточка (Т-2) - Абдисаламова Бактыгул (Сотрудник) - 14.02.2022.xlsx
Личная карточка (Т-2) - Аношкин Андрей Иванович (Сотрудник) - 14.02.2022.xlsx
Личная карточка (Т-2) - Антипова Александра Михайловна (Сотрудник) - 14.02.2022.xlsx
Личная карточка (Т-2) - Бачало Наталья Тимофеевна (Сотрудник) - 14.02.2022.xlsx
Личная карточка (Т-2) - Безунов Игорь Андреевич (Сотрудник) - 14.02.2022.xlsx
Личная карточка (Т-2) - Беспалов Дмитрий Дмитриевич (Сотрудник) - 14.02.2022.xlsx
Личная карточка (Т-2) - Бешлый Ярослав Владимирович (Сотрудник) - 14.02.2022.xlsx
Личная карточка (Т-2) - Васяева Ольга Анатольевна (Сотрудник) - 14.02.2022.xlsx
Личная карточка (Т-2) - Витковский Алексей Николаевич (Сотрудник) - 14.02.2022.xlsx
Личная карточка (Т-2) - Гребенщиков Никита Андреевич (Сотрудник) - 14.02.2022.xlsx
Личная карточка (Т-2) - Грибанова Алла Ивановна (Сотрудник) - 14.02.2022.xlsx
Личная карточка (Т-2) - Даньшина Анна Григорьевна (Сотрудник) - 14.02.2022.xlsx
Личная карточка (Т-2) - Джинджихад

,Пол,ФИО,Возраст,Стаж (в днях),Образование,Наименование образовательного учреждения,Квалификация по документу об образовании,Направление или специальность по документу,Должность,Отдел,"Должность занималась, дней",Состояние в браке,Количество детей,Минимальный возраст детей,Средний возраст детей,Максимальный возраст детей,Адрес места жительства,Дата увольнения,"Уволен (1 да, 0 нет)"
0,Женский,Абдисаламова Бактыгул,23,36,None,None,None,None,Уборщик производственных и служебных помещений,015.4. ОБЩЕЖИТИЕ,36,None,0,0,0,0,"РОССИЯ, Москва г, муниципальный округ Ясенево...",14.08.2020,1
1,Женский,Абдисаламова Бактыгул,23,36,None,None,None,None,Уборщик помещений,ОТДЕЛ АДМИНИСТРАТИВНО-ХОЗЯЙСТВЕННОГО ОБЕСПЕЧЕНИЯ,13,None,0,0,0,0,"РОССИЯ, Москва г, муниципальный округ Ясенево...",14.08.2020,1
2,Мужской,Аношкин Андрей Иванович,58,537,"Высшее образование - специалитет, магистратура",Дальневосточный государственный технический ры...,Менеджер,Государственное и муниципальное управление,Заместитель начальника отдела,015. ОТДЕЛ ОБЕСПЕЧЕНИЯ ДЕЯТЕЛЬНОСТИ УЧРЕЖДЕНИЯ,537,None,1,25,25,25,"РОССИЯ, Москва г, Таможенный проезд, дом № 8,...",27.03.2020,1
3,Женский,Антипова Александра Михайловна,30,798,"Высшее образование - специалитет, магистратура","ГБОУ ВПО ""Ярославская государственная медицинс...",врач,инфекционные болезни,Врач клинической лабораторной диагностики,002.4. ЛАБОРАТОРИЯ ОСОБО ОПАСНЫХ ВИРУСНЫХ ИНФ...,798,Никогда не состоял(а) в браке,0,0,0,0,"РОССИЯ, Костромская обл, Кострома г, Ленина у...",27.05.2020,1
4,Женский,Бачало Наталья Тимофеевна,26,222,Высшее образование - бакалавриат,Московский государственный университет техноло...,Бакалавр,Продукты питания из растительного сырья,Инженер,ЛАБОРАТОРИЯ САНИТАРНО-ХИМИЧЕСКИХ И ТОКСИКОЛОГИ...,222,Состоит в зарегистрированном браке,0,0,0,0,"РОССИЯ, Москва г, Внутригородская территория ...",25.09.2020,1
5,Мужской,Безунов Игорь Андреевич,24,510,"Высшее образование - специалитет, магистратура",ФГАОУ ВО Первый Московский госудаственный меди...,"Врач по общей гигиене, по эпидемиологии",Медико-профилактическое дело,Помощник врача по гигиене труда,ОТДЕЛ ОБЕСПЕЧЕНИЯ САНИТАРНОГО НАДЗОРА,510,None,0,0,0,0,"РОССИЯ, Москва г, Затонная, дом № 14, корпус ...",28.10.2020,1
6,Мужской,Безунов Игорь Андреевич,24,510,"Высшее образование - специалитет, магистратура",ФГАОУ ВО Первый Московский госудаственный меди...,"Врач по общей гигиене, по эпидемиологии",Медико-профилактическое дело,Помощник врача по гигиене труда,ОТДЕЛ ОБЕСПЕЧЕНИЯ САНИТАРНОГО НАДЗОРА,482,None,0,0,0,0,"РОССИЯ, Москва г, Затонная, дом № 14, корпус ...",28.10.2020,1
7,Мужской,Безунов Игорь Андреевич,24,510,"Высшее образование - специалитет, магистратура",ФГАОУ ВО Первый Московский госудаственный меди...,"Врач по общей гигиене, по эпидемиологии",Медико-профилактическое дело,Помощник врача по гигиене труда,ОТДЕЛ ОБЕСПЕЧЕНИЯ САНИТАРНОГО НАДЗОРА,421,None,0,0,0,0,"РОССИЯ, Москва г, Затонная, дом № 14, корпус ...",28.10.2020,1
8,Мужской,Безунов Игорь Андреевич,24,510,"Высшее образование - специалитет, магистратура",ФГАОУ ВО Первый Московский госудаственный меди...,"Врач по общей гигиене, по эпидемиологии",Медико-профилактическое дело,Врач по общей гигиене,ОТДЕЛ ОБЕСПЕЧЕНИЯ САНИТАРНОГО НАДЗОРА,102,None,0,0,0,0,"РОССИЯ, Москва г, Затонная, дом № 14, корпус ...",28.10.2020,1
9,Мужской,Беспалов Дмитрий Дмитриевич,25,984,Высшее образование,ГОУВПО ММА им.И.М.Сеченова,"врач по общей гигиене, по эпидемиологии",Медико-профилактическое дело,Помощник врача по гигиене питания,004.3. ОТДЕЛЕНИЕ ОБЕСПЕЧЕНИЯ САНИТАРНОГО НАДЗО...,984,Состоит в зарегистрированном браке,1,2,2,2,"Московская обл, Подольск г, Чайковского ул, д...",14.08.2020,1


# обработка уволенных 2021

In [35]:
os.chdir(path_fired_processed_2021)
files = remove_formats(os.listdir(), format='.xlsx')

to_df = make_empty_dict()
df_fired_2021 = pd.DataFrame(columns=list(to_df.keys()))
for _ in files:
    wb = load_workbook(_)
    sheet = wb.active
    print(_)
    employee = {}
    childrens, birth_year, structure, date, position = [], [], [], [], []
    for row in sheet['A1':'AG300']: # движение по строкам в диапазоне ['A1':'AG300']
        for cell in row: # движение по ячейкам в строке row
            for ex in list_of_all: # перебор значений из списка того, что нужно выдрать из анкет
                if cell.value == ex[0]: # сравнение значения ячейки с названием нужного "столбца"
                    if len(ex[1]) == 1: # проверяем "направление" на движение в одной плоскости
                        if ex[1][0] == 'down': # от целевой ячейки движение вниз на ex[2][0]
                            if sheet.cell(row=cell.row+ex[2][0], column=cell.col_idx).value == None:
                                break
                            employee[str(ex[0])] = sheet.cell(row=cell.row+ex[2][0], column=cell.col_idx).value
                        elif ex[1][0] == 'right': # от целевой ячейки движение вправо на ex[2][0]
                            if sheet.cell(row=cell.row, column=cell.col_idx+ex[2][0]).value == None:
                                break
                            employee[str(ex[0])] = sheet.cell(row=cell.row, column=cell.col_idx+ex[2][0]).value
                    elif len(ex[1]) > 1: # от целевой клетки движение на ex[2][0] вниз и на ex[2][1] вправо
                        if sheet.cell(row=cell.row+ex[2][0], column=cell.col_idx+ex[2][1]).value == None:
                            break
                        employee[str(ex[0])] = sheet.cell(row=cell.row+ex[2][0], column=cell.col_idx+ex[2][1]).value
                    if '10. Состав семьи:' in cell.value: # исключение для детишек, просто ебанутый случай, нужно выдрать list[]
                        for i in list(range(1,7)):
                            if sheet.cell(row=cell.row+2+i, column=cell.col_idx).value == None:
                                break
                            childrens.append(sheet.cell(row=cell.row+2+i, column=cell.col_idx).value)
                            birth_year.append(sheet.cell(row=cell.row+2+i, column=cell.col_idx+30).value)
                    if 'Структурное\nподразделение' in cell.value: # высасываем отдел-должность в list[]
                        for i in list(range(10)):
                            if sheet.cell(row=cell.row+i+2, column=cell.col_idx).value == None:
                                break
                            structure.append(sheet.cell(row=cell.row+i+2, column=cell.col_idx).value)
                            date.append(sheet.cell(row=cell.row+i+2, column=cell.col_idx-1).value)
                            position.append(sheet.cell(row=cell.row+i+2, column=cell.col_idx+5).value)
    employee.update({'childrens':childrens, 'birth_year':birth_year, 'structure':structure, 'date':date,'position':position,
                     '2. Дата рождения':date_to_str(employee['2. Дата рождения']),
                      'Дата увольнения': date_to_str(employee['Дата увольнения'])})
       
    try:
        fio = fio_enhance(employee['1. Фамилия'], employee['Имя'], employee['Отчество'])
    except KeyError:
        fio = fio_enhance(employee['1. Фамилия'], employee['Имя'])
    employee['fio'] = fio
    employee = make_missing_keys(employee)
    
    for i in range(len(employee['structure'])):
        to_df.update({'Пол':[employee['Пол\n(мужской,\nженский)']],
                 'ФИО':[employee['fio']],
                 'Возраст':[date_age(current_date(), employee['2. Дата рождения'])],
                 'Стаж (в днях)':[date_delta(employee['Дата увольнения'], employee['Трудовой договор'])],
                 'Образование':[employee['6. Образование']],
                 'Наименование образовательного учреждения':[employee['Наименование образовательного учреждения']],
                 'Квалификация по документу об образовании':[employee['Квалификация по документу об образовании']],
                 'Направление или специальность по документу':[employee['Направление или специальность по документу']],
                 'Должность':[employee['position'][i]],
                 'Отдел':[employee['structure'][i]],
                 'Должность занималась, дней':[date_delta(date_to_str(employee['Дата увольнения']), date_to_str(employee['date'][i]))],
                 'Состояние в браке':[employee['9. Состояние в браке']],
                 'Количество детей':[pererabotka_detey(employee['childrens'], employee['birth_year'])[0]],
                 'Минимальный возраст детей':[pererabotka_detey(employee['childrens'], employee['birth_year'])[1]],
                 'Средний возраст детей':[pererabotka_detey(employee['childrens'], employee['birth_year'])[2]],
                 'Максимальный возраст детей':[pererabotka_detey(employee['childrens'], employee['birth_year'])[3]],
                 'Адрес места жительства':[employee['Фактический']],
                 'Дата увольнения':[date_to_str(employee['Дата увольнения'])],
                 })

        to_append = pd.DataFrame(data=to_df)
        df_fired_2021 = pd.concat([df_fired_2021, to_append], ignore_index=True, axis=0)
df_fired_2021['Уволен (1 да, 0 нет)'] = 1
df_fired_2021

Личная карточка (Т-2) - Алексеева Галина Николаевна (Сотрудник) - 14.02.2022.xlsx
Личная карточка (Т-2) - Базилевская Елена Константиновна (Сотрудник) - 14.02.2022.xlsx
Личная карточка (Т-2) - Белушенко Антон Олегович (Сотрудник) - 14.02.2022.xlsx
Личная карточка (Т-2) - Беляев Евгений Николаевич (Сотрудник) - 14.02.2022.xlsx
Личная карточка (Т-2) - Воронова Мария Викторовна (Сотрудник) - 14.02.2022.xlsx
Личная карточка (Т-2) - Григорьев Дмитрий Михайлович (Сотрудник) - 14.02.2022.xlsx
Личная карточка (Т-2) - Давлетшина Альбина Рашитовна  (Сотрудник) - 14.02.2022.xlsx
Личная карточка (Т-2) - Дворнов Евгений Владимирович (Сотрудник) - 14.02.2022.xlsx
Личная карточка (Т-2) - Крамаренко Анна Александровна (Сотрудник) - 14.02.2022.xlsx
Личная карточка (Т-2) - Локтаева Ольга Сергеевна (Сотрудник) - 14.02.2022.xlsx
Личная карточка (Т-2) - Макурина Анна Юрьевна (Сотрудник) - 14.02.2022.xlsx
Личная карточка (Т-2) - Мотылева Марина Викторовна (Сотрудник) - 14.02.2022.xlsx
Личная карточка (Т-2) 

,Пол,ФИО,Возраст,Стаж (в днях),Образование,Наименование образовательного учреждения,Квалификация по документу об образовании,Направление или специальность по документу,Должность,Отдел,"Должность занималась, дней",Состояние в браке,Количество детей,Минимальный возраст детей,Средний возраст детей,Максимальный возраст детей,Адрес места жительства,Дата увольнения,"Уволен (1 да, 0 нет)"
0,Женский,Алексеева Галина Николаевна,61,5085,None,None,None,None,Дежурная по общежитию,1125. ОБЩЕЖИТИЕ ГОСТИНИЧНОГО ТИПА,5170,Никогда не состоял(а) в браке,1,8,8,8,"Москва г, Харьковский проезд, дом № 11, корпу...",1.03.2021,1
1,Женский,Алексеева Галина Николаевна,61,5085,None,None,None,None,Дежурная по общежитию,1211. ОБЩЕЖИТИЕ,5085,Никогда не состоял(а) в браке,1,8,8,8,"Москва г, Харьковский проезд, дом № 11, корпу...",1.03.2021,1
2,Женский,Алексеева Галина Николаевна,61,5085,None,None,None,None,Дежурная по общежитию,1211. ОБЩЕЖИТИЕ,4805,Никогда не состоял(а) в браке,1,8,8,8,"Москва г, Харьковский проезд, дом № 11, корпу...",1.03.2021,1
3,Женский,Алексеева Галина Николаевна,61,5085,None,None,None,None,Дежурная по общежитию,1211. ОБЩЕЖИТИЕ,4475,Никогда не состоял(а) в браке,1,8,8,8,"Москва г, Харьковский проезд, дом № 11, корпу...",1.03.2021,1
4,Женский,Алексеева Галина Николаевна,61,5085,None,None,None,None,Дежурная по общежитию,1211. ОБЩЕЖИТИЕ,3560,Никогда не состоял(а) в браке,1,8,8,8,"Москва г, Харьковский проезд, дом № 11, корпу...",1.03.2021,1
5,Женский,Алексеева Галина Николаевна,61,5085,None,None,None,None,Дежурная по общежитию,1211. ОБЩЕЖИТИЕ,3440,Никогда не состоял(а) в браке,1,8,8,8,"Москва г, Харьковский проезд, дом № 11, корпу...",1.03.2021,1
6,Женский,Алексеева Галина Николаевна,61,5085,None,None,None,None,Дежурная по общежитию,1211. ОБЩЕЖИТИЕ,3075,Никогда не состоял(а) в браке,1,8,8,8,"Москва г, Харьковский проезд, дом № 11, корпу...",1.03.2021,1
7,Женский,Алексеева Галина Николаевна,61,5085,None,None,None,None,Дежурная по общежитию,1211. ОБЩЕЖИТИЕ,2980,Никогда не состоял(а) в браке,1,8,8,8,"Москва г, Харьковский проезд, дом № 11, корпу...",1.03.2021,1
8,Женский,Алексеева Галина Николаевна,61,5085,None,None,None,None,Дежурная по общежитию,1211. ОБЩЕЖИТИЕ,2710,Никогда не состоял(а) в браке,1,8,8,8,"Москва г, Харьковский проезд, дом № 11, корпу...",1.03.2021,1
9,Женский,Алексеева Галина Николаевна,61,5085,None,None,None,None,Дежурная по общежитию,015.4. ОБЩЕЖИТИЕ,1249,Никогда не состоял(а) в браке,1,8,8,8,"Москва г, Харьковский проезд, дом № 11, корпу...",1.03.2021,1


# обработка нынешних

In [36]:
os.chdir(path_processed)
files = remove_formats(os.listdir(), format='.xlsx')

to_df = make_empty_dict()
df_curr = pd.DataFrame(columns=list(to_df.keys()))
for _ in files:
    wb = load_workbook(_)
    sheet = wb.active
    print(_)
    employee = {}
    childrens, birth_year, structure, date, position = [], [], [], [], []
    for row in sheet['A1':'AG300']: # движение по строкам в диапазоне ['A1':'AG300']
        for cell in row: # движение по ячейкам в строке row
            for ex in list_of_all: # перебор значений из списка того, что нужно выдрать из анкет
                if cell.value == ex[0]: # сравнение значения ячейки с названием нужного "столбца"
                    if len(ex[1]) == 1: # проверяем "направление" на движение в одной плоскости
                        if ex[1][0] == 'down': # от целевой ячейки движение вниз на ex[2][0]
                            if sheet.cell(row=cell.row+ex[2][0], column=cell.col_idx).value == None:
                                break
                            employee[str(ex[0])] = sheet.cell(row=cell.row+ex[2][0], column=cell.col_idx).value
                        elif ex[1][0] == 'right': # от целевой ячейки движение вправо на ex[2][0]
                            if sheet.cell(row=cell.row, column=cell.col_idx+ex[2][0]).value == None:
                                break
                            employee[str(ex[0])] = sheet.cell(row=cell.row, column=cell.col_idx+ex[2][0]).value
                    elif len(ex[1]) > 1: # от целевой клетки движение на ex[2][0] вниз и на ex[2][1] вправо
                        if sheet.cell(row=cell.row+ex[2][0], column=cell.col_idx+ex[2][1]).value == None:
                            break
                        employee[str(ex[0])] = sheet.cell(row=cell.row+ex[2][0], column=cell.col_idx+ex[2][1]).value
                    if '10. Состав семьи:' in cell.value: # исключение для детишек, просто ебанутый случай, нужно выдрать list[]
                        for i in list(range(1,7)):
                            if sheet.cell(row=cell.row+2+i, column=cell.col_idx).value == None:
                                break
                            childrens.append(sheet.cell(row=cell.row+2+i, column=cell.col_idx).value)
                            birth_year.append(sheet.cell(row=cell.row+2+i, column=cell.col_idx+30).value)
                    if 'Структурное\nподразделение' in cell.value: # высасываем отдел-должность в list[]
                        for i in list(range(10)):
                            if sheet.cell(row=cell.row+i+2, column=cell.col_idx).value == None:
                                break
                            structure.append(sheet.cell(row=cell.row+i+2, column=cell.col_idx).value)
                            date.append(sheet.cell(row=cell.row+i+2, column=cell.col_idx-1).value)
                            position.append(sheet.cell(row=cell.row+i+2, column=cell.col_idx+5).value)
    employee.update({'childrens':childrens, 'birth_year':birth_year, 'structure':structure, 'date':date,'position':position,
                     '2. Дата рождения':date_to_str(employee['2. Дата рождения']),
                      'Дата увольнения': date_to_str(employee['Дата увольнения'])})
       
    try:
        fio = fio_enhance(employee['1. Фамилия'], employee['Имя'], employee['Отчество'])
    except KeyError:
        fio = fio_enhance(employee['1. Фамилия'], employee['Имя'])
    employee['fio'] = fio
    employee = make_missing_keys(employee)
    
    for i in range(len(employee['structure'])):
        to_df.update({'Пол':[employee['Пол\n(мужской,\nженский)']],
                 'ФИО':[employee['fio']],
                 'Возраст':[date_age(current_date(), employee['2. Дата рождения'])],
                 'Стаж (в днях)':[date_delta(current_date(), employee['Трудовой договор'])],
                 'Образование':[employee['6. Образование']],
                 'Наименование образовательного учреждения':[employee['Наименование образовательного учреждения']],
                 'Квалификация по документу об образовании':[employee['Квалификация по документу об образовании']],
                 'Направление или специальность по документу':[employee['Направление или специальность по документу']],
                 'Должность':[employee['position'][i]],
                 'Отдел':[employee['structure'][i]],
                 'Должность занималась, дней':[date_delta(date_to_str(current_date()), date_to_str(employee['date'][i]))],
                 'Состояние в браке':[employee['9. Состояние в браке']],
                 'Количество детей':[pererabotka_detey(employee['childrens'], employee['birth_year'])[0]],
                 'Минимальный возраст детей':[pererabotka_detey(employee['childrens'], employee['birth_year'])[1]],
                 'Средний возраст детей':[pererabotka_detey(employee['childrens'], employee['birth_year'])[2]],
                 'Максимальный возраст детей':[pererabotka_detey(employee['childrens'], employee['birth_year'])[3]],
                 'Адрес места жительства':[employee['Фактический']],
                 'Дата увольнения':[0],
                 })

        to_append = pd.DataFrame(data=to_df)
        df_curr = pd.concat([df_curr, to_append], ignore_index=True, axis=0)
df_curr['Уволен (1 да, 0 нет)'] = 0
df_curr

Личная карточка (Т-2) - Алиярова Гюлселем Рамисовна (Сотрудник) - 25.01.2022.xlsx
Личная карточка (Т-2) - Андросова Ольга Викторовна (Сотрудник) - 25.01.2022.xlsx
Личная карточка (Т-2) - Антонов Дмитрий Владиславович (Сотрудник) - 25.01.2022.xlsx
Личная карточка (Т-2) - Ануфриева Елена Владимировна (Сотрудник) - 25.01.2022.xlsx
Личная карточка (Т-2) - Анцупова Елена Николаевна (Сотрудник) - 25.01.2022.xlsx
Личная карточка (Т-2) - Асадова Анастасия Александровна (Сотрудник) - 25.01.2022.xlsx
Личная карточка (Т-2) - Ахунова Назигуль Рахманбердиевна (Сотрудник) - 25.01.2022.xlsx
Личная карточка (Т-2) - Бабина Александра Павловна (Сотрудник) - 25.01.2022.xlsx
Личная карточка (Т-2) - Бакаев Сергей Евгеньевич (Сотрудник) - 25.01.2022.xlsx
Личная карточка (Т-2) - Бакалова Надежда Юрьевна (Сотрудник) - 25.01.2022.xlsx
Личная карточка (Т-2) - Бегларян Ашот Рубенович (Сотрудник) - 25.01.2022.xlsx
Личная карточка (Т-2) - Беликова Янина Александровна (Сотрудник) - 25.01.2022.xlsx
Личная карточка (

Личная карточка (Т-2) - Малян Елена Эдуардовна (Сотрудник) - 25.01.2022.xlsx
Личная карточка (Т-2) - Маркушева Ольга Владимировна (Сотрудник) - 25.01.2022.xlsx
Личная карточка (Т-2) - Матвеева Виктория Андреевна (Сотрудник) - 25.01.2022.xlsx
Личная карточка (Т-2) - Медведь Евгения Дмитриевна (Сотрудник) - 25.01.2022.xlsx
Личная карточка (Т-2) - Миронова Ирина Игоревна (Сотрудник) - 25.01.2022.xlsx
Личная карточка (Т-2) - Митрофанова Екатерина Сергеевна (Сотрудник) - 25.01.2022.xlsx
Личная карточка (Т-2) - Михайлова Юлия Михайловна (Сотрудник) - 25.01.2022.xlsx
Личная карточка (Т-2) - Мокренская Светлана Алексеевна (Сотрудник) - 25.01.2022.xlsx
Личная карточка (Т-2) - Моргачёв Олег Васильевич (Сотрудник) - 25.01.2022.xlsx
Личная карточка (Т-2) - Мордвинова Валерия Вячеславовна (Сотрудник) - 25.01.2022.xlsx
Личная карточка (Т-2) - Морозова Надежда Сергеевна (Сотрудник) - 25.01.2022.xlsx
Личная карточка (Т-2) - Мурадова Улькер Эльман кызы (Сотрудник) - 25.01.2022.xlsx
Личная карточка (Т-2

,Пол,ФИО,Возраст,Стаж (в днях),Образование,Наименование образовательного учреждения,Квалификация по документу об образовании,Направление или специальность по документу,Должность,Отдел,"Должность занималась, дней",Состояние в браке,Количество детей,Минимальный возраст детей,Средний возраст детей,Максимальный возраст детей,Адрес места жительства,Дата увольнения,"Уволен (1 да, 0 нет)"
0,Женский,Алиярова Гюлселем Рамисовна,32,1176,"Высшее образование - специалитет, магистратура","ФГБОУ ВПО ""Дагестанский государственный универ...",Химик,Химия,Химик-эксперт,002.1.1. ОТДЕЛЕНИЕ ФИЗИКО - ХИМИЧЕСКИХ ИССЛЕДО...,1176,Состоит в зарегистрированном браке,0,0,0,0,"РОССИЯ, Москва г, Лебедянская ул, дом 22, кор...",0,0
1,Женский,Алиярова Гюлселем Рамисовна,32,1176,"Высшее образование - специалитет, магистратура","ФГБОУ ВПО ""Дагестанский государственный универ...",Химик,Химия,Химик-эксперт,ЛАБОРАТОРИЯ ФИЗИКО-ХИМИЧЕСКИХ ИССЛЕДОВАНИЙ,1130,Состоит в зарегистрированном браке,0,0,0,0,"РОССИЯ, Москва г, Лебедянская ул, дом 22, кор...",0,0
2,Женский,Андросова Ольга Викторовна,53,1562,Высшее образование,Московкий ин-т тонкой химической технологии им...,инженер технолог,Биотехнология,Зав.лабораторией-врач-лаборант,1025. ЛАБОРАТОРИЯ ФИЗИКО-ХИМИЧЕСКИХ ИССЛЕДОВАНИЙ,1562,None,0,0,0,0,"Москва г, Южнобутовская ул, дом № 25, корпус ...",0,0
3,Женский,Андросова Ольга Викторовна,53,1562,Высшее образование,Московкий ин-т тонкой химической технологии им...,инженер технолог,Биотехнология,Зав.лабораторией-врач-лаборант,002.1. САНИТАРНО-ХИМИЧЕСКАЯ ЛАБОРАТОРИЯ,1548,None,0,0,0,0,"Москва г, Южнобутовская ул, дом № 25, корпус ...",0,0
4,Женский,Андросова Ольга Викторовна,53,1562,Высшее образование,Московкий ин-т тонкой химической технологии им...,инженер технолог,Биотехнология,Заместитель заведующего отделом - врач-лаборант,002. ОТДЕЛ ЛАБОРАТОРНОГО ДЕЛА,1130,None,0,0,0,0,"Москва г, Южнобутовская ул, дом № 25, корпус ...",0,0
5,Мужской,Антонов Дмитрий Владиславович,58,298,Высшее образование,Московский технический университет связи и инф...,Инженер,Многоканальные телекоммуникационные системы,Руководитель службы,АДМИНИСТРАТИВНО-ХОЗЯЙСТВЕННАЯ СЛУЖБА,298,Разведен(а) официально (развод зарегистрирован),1,12,12,12,"РОССИЯ, Московская обл, г Химки, ул Зеленая, ...",0,0
6,Мужской,Антонов Дмитрий Владиславович,58,298,Высшее образование,Московский технический университет связи и инф...,Инженер,Многоканальные телекоммуникационные системы,Инженер,ОТДЕЛ АДМИНИСТРАТИВНО-ХОЗЯЙСТВЕННОГО ОБЕСПЕЧЕНИЯ,42,Разведен(а) официально (развод зарегистрирован),1,12,12,12,"РОССИЯ, Московская обл, г Химки, ул Зеленая, ...",0,0
7,Женский,Ануфриева Елена Владимировна,41,118,Высшее образование,"ГОУ ВПО ""Уральская государственная медицинская...",Врач,Лечебное дело,Редактор,УЧЕБНО-ИЗДАТЕЛЬСКИЙ ОТДЕЛ,118,None,0,0,0,0,"РОССИЯ, Свердловская обл, г Екатеринбург, ул ...",0,0
8,Женский,Анцупова Елена Николаевна,29,1962,Высшее образование,ГБОУ ВПО Первый МГМУ им.И.М.Сеченова,Врач,Медико-профилактическое дело,Помощник врача по общей гигиене,1012. ОТДЕЛЕНИЕ СТАТИСТИЧЕСКОГО НАБЛЮДЕНИЯ И С...,1962,None,0,0,0,0,"РОССИЯ, Москва г, Медынская ул, дом № 8, корп...",0,0
9,Женский,Анцупова Елена Николаевна,29,1962,Высшее образование,ГБОУ ВПО Первый МГМУ им.И.М.Сеченова,Врач,Медико-профилактическое дело,Врач по общей гигиене,1012. ОТДЕЛЕНИЕ СТАТИСТИЧЕСКОГО НАБЛЮДЕНИЯ И С...,1629,None,0,0,0,0,"РОССИЯ, Москва г, Медынская ул, дом № 8, корп...",0,0


In [38]:
df_fired_2018.shape, df_fired_2019.shape, df_fired_2020.shape, df_fired_2021.shape, df_curr.shape

((281, 19), (248, 19), (111, 19), (88, 19), (886, 19))

In [42]:
df = pd.concat([df, df_fired_2018], ignore_index=True, axis=0)
df.shape

(1614, 19)

In [31]:
#df = pd.concat([df, df_curr], ignore_index=True, axis=0)
df.shape

(974, 19)

In [43]:
os.chdir('C:/Users/zhilapv/Desktop/HR_analysis')
df.to_excel('df_3.1.xlsx')